<a href="https://colab.research.google.com/github/Gowri-Rk/Machine-Learning/blob/master/Colabs/Models/ML_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing
from sklearn import utils

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:

df_exams = pd.read_csv('/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/Datasets/Exam_Cleaned.csv', encoding='unicode_escape')

In [ ]:
df_exams.head()

,Unnamed: 0,Unnamed: 0.1,SEQN,PEASCST1,PEASCTM1,BPXPLS,BPXPULS,BPXML1,BPXSY1,BPXDI1,BPXSY2,BPXDI2,BPXSY3,BPXDI3,BMXWT,BMXHT,BMXBMI,BMDAVSAD,MGDEXSTS
0,0,0,73557,1,620.0,86.0,1.0,140.0,122.0,72.0,114.0,76.0,102.0,74.0,78.3,171.3,26.7,20.600000,1.0
1,1,1,73558,1,766.0,74.0,1.0,170.0,156.0,62.0,160.0,80.0,156.0,42.0,89.5,176.8,28.6,24.400000,1.0
2,2,2,73559,1,665.0,68.0,1.0,160.0,140.0,90.0,140.0,76.0,146.0,80.0,88.9,175.3,28.9,25.600000,1.0
3,3,3,73560,1,803.0,64.0,1.0,130.0,108.0,38.0,102.0,34.0,104.0,38.0,32.2,137.3,17.1,14.900000,1.0
4,4,4,73561,1,949.0,92.0,1.0,170.0,136.0,86.0,134.0,88.0,142.0,86.0,52.0,162.4,19.7,21.114034,1.0


In [ ]:
df_labs2 = pd.read_csv('/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/Datasets/Reduced_labs_cleaned.csv', encoding='unicode_escape')

In [ ]:
df_labs2.head()

,Unnamed: 0,SEQN,URXUMA,URXUMS,URXUCR.x,URXCRS,URDACT,WTSAF2YR.x,LBDAPBSI,LBXSAL,LBDSALSI,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBDSBUSI,LBXSC3SI,LBXSCA,LBDSCASI,LBXSCH,LBDSCHSI,LBXSCK,LBXSCLSI,LBXSCR,LBDSCRSI,LBXSGB,LBDSGBSI,LBXSGL,LBDSGLSI,LBXSGTSI,LBXSIR,LBDSIRSI,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBDSTBSI,...,PHAFSTHR.y,PHAFSTMN.y,URXUIO,WTSAF2YR,LBXTR,LBDTRSI,LBDLDL,LBDLDLSI,WTSH2YR.x,WTSOG2YR,LBXGLT,LBDGLTSI,GTDSCMMN,GTDDR1MN,GTDBL2MN,GTDDR2MN,GTXDRANK,PHAFSTHR,PHAFSTMN,WTSA2YR.y,LBXTC,LBDTCSI,LBXBPB,LBDBPBSI,LBDBPBLC,LBXBCD,LBDBCDSI,LBDBCDLC,LBXTHG,LBDTHGSI,LBDTHGLC,LBXBSE,LBDBSESI,LBDBSELC,LBXBMN,LBDBMNSI,LBDBMNLC,URXVOL1,LBDB12,LBDB12SI
0,0,73557,4.3,4.3,39.0,3447.6,11.03,NaN,NaN,4.1,41.0,129.0,16.0,16.0,10.0,3.57,27.0,9.5,2.375,168.0,4.344,174.0,97.0,1.21,106.96,2.4,24.0,554.0,30.75,16.0,58.0,10.4,4.3,118.0,136.0,296.0,4.3,1.388,0.8,13.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,4.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0,524.0,386.7
1,1,73558,153.0,153.0,50.0,4420.0,306.00,NaN,NaN,4.7,47.0,97.0,18.0,29.0,16.0,5.71,23.0,9.2,2.300,167.0,4.319,147.0,98.0,0.79,69.84,3.1,31.0,219.0,12.16,15.0,79.0,14.1,4.3,122.0,128.0,265.0,3.9,1.259,0.9,15.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49123.400015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170.0,4.40,1.69,0.082,0.0,0.20,1.78,0.0,1.21,6.0,0.0,204.73,2.60,0.0,8.15,148.33,0.0,90.0,507.0,374.2
2,2,73559,11.9,11.9,113.0,9989.2,10.53,142196.890197,0.57,3.7,37.0,99.0,22.0,16.0,14.0,5.00,23.0,8.9,2.225,127.0,3.284,44.0,105.0,1.22,107.85,2.2,22.0,183.0,10.16,13.0,98.0,17.6,4.2,134.0,142.0,288.0,4.2,1.356,0.6,10.26,...,15.0,51.0,NaN,142196.890197,51.0,0.576,56.0,1.448,115794.742161,143745.934346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,51.0,NaN,126.0,3.26,1.45,0.070,0.0,0.22,1.96,0.0,1.27,6.3,0.0,209.64,2.66,0.0,9.57,174.17,0.0,66.0,732.0,540.2
3,3,73560,16.0,16.0,76.0,6718.4,21.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,194.3,NaN,NaN,NaN,NaN,NaN,55766.512438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183653.604036,168.0,4.34,0.37,0.018,0.0,0.07,0.62,1.0,0.88,4.4,0.0,169.82,2.16,0.0,13.07,237.87,0.0,61.0,NaN,NaN
4,4,73561,255.0,255.0,147.0,12994.8,173.47,142266.006548,0.92,4.3,43.0,78.0,36.0,28.0,31.0,11.07,31.0,10.0,2.500,207.0,5.353,60.0,100.0,0.73,64.53,2.8,28.0,104.0,5.77,31.0,91.0,16.3,4.1,201.0,142.0,290.0,4.4,1.421,0.5,8.55,...,14.0,42.0,NaN,142266.006548,75.0,0.847,101.0,2.612,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,3.0,14.0,42.0,NaN,201.0,5.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,225.0,166.1


**Refining the dataset**

In [ ]:
newX = pd.merge(df_labs2, df_exams, on='SEQN', how='inner')

In [ ]:
newX = newX[['BMXBMI', 'LBDINSI']]

In [ ]:
#REmoving outliers
low = .05
high = .99
quant_df = newX.quantile([low, high])
print(quant_df)

      BMXBMI   LBDINSI
0.05  15.300   18.7800
0.99  49.864  459.7776


In [ ]:
#removing outliers
newX = newX.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)

In [ ]:
newX

,BMXBMI,LBDINSI
0,26.7,NaN
1,28.6,NaN
2,28.9,34.98
3,17.1,NaN
4,19.7,36.72
...,...,...
9808,24.5,22.56
9809,15.9,NaN
9810,34.0,NaN
9811,16.1,NaN


In [ ]:
X1 = newX['BMXBMI']

In [ ]:
newX.isna().sum()

BMXBMI       58
LBDINSI    6328
dtype: int64

In [ ]:
newX['LBDINSI'] = newX['LBDINSI'].fillna(newX['LBDINSI'].mean())
newX['BMXBMI'] = newX['BMXBMI'].fillna(newX['BMXBMI'].mean())

In [ ]:
newX.isna().sum()

BMXBMI     0
LBDINSI    0
dtype: int64

In [ ]:
X1 = newX['BMXBMI']

In [ ]:
Y1 = newX['LBDINSI']

**Encoding feature to binary**

In [ ]:
enc = preprocessing.LabelEncoder()
encodedY1 = enc.fit_transform(Y1)

In [ ]:
Y1PD = pd.DataFrame(encodedY1)

**Performing Logistic Regression**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1PD, test_size=0.25, random_state=0)

In [ ]:
#reshape
X_train = X_train.values.reshape(-1,1)

In [ ]:
y_train = y_train.values.reshape(-1,1)

In [ ]:
X_test = X_test.values.reshape(-1,1)


In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
logreg = LogisticRegression(random_state=0, max_iter=1000)

In [ ]:
# Train the model using the training sets
logreg.fit(X_train, y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred=logreg.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6829796448679082
